In [1]:
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf

In [2]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    test_dataset = save['test_dataset']
    train_old = save['t_v_dataset']
    del save


test_data_1 = test_dataset['data_ASOS']
test_data_2 = test_dataset['data_NWP']
test_label = test_dataset['label']
train_data_1 = train_old['data_ASOS']
train_data_2 = train_old['data_NWP']
train_label = train_old['label']

In [5]:
train_data = np.hstack((train_data_1, train_data_2))
test_data = np.hstack((test_data_1, test_data_2))
train_data.shape, test_data.shape

((116657, 82), (29165, 82))

In [6]:
#dataset normalize
mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
print(mean.shape, std.shape)
train_data_n = (train_data - mean)/std
test_data_n = (test_data - mean)/std

(82,) (82,)


Transfering the regression problem to classification problem

In [7]:
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(train_label.reshape(1, -1))
b_test_label = pre.transform(test_label.reshape(1, -1))

In [8]:
c_train_label = 1 - b_train_label[0]

c_test_label = 1 - b_test_label[0]

In [10]:
#shuffle the data set
arr = np.arange(c_train_label.shape[0])
np.random.shuffle(arr)
train_data_n_s =  train_data_n[arr]
c_train_label_s = c_train_label[arr]

In [25]:
#need to balanced the dataset
from unbalanced_dataset.over_sampling import SMOTE
sm = SMOTE(ratio = 0.03, kind='regular')
train_data_n_resample, c_train_label_resample = sm.fit_transform(train_data_n, c_train_label)

Determining classes statistics... 2 classes detected: Counter({0.0: 114677, 1.0: 1980})
Finding the 5 nearest neighbours...
done!
Creating synthetic samples...Generated 1460 new samples ...
done!


In [31]:
#shuffle the data set
arr = np.arange(c_train_label_resample.shape[0])
np.random.shuffle(arr)
train_data_n_resample_s =  train_data_n_resample[arr]
c_train_label_resample_s = c_train_label_resample[arr]

In [11]:
c_train_label_s[:100]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [12]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [17]:
v_train_label_s = makeIndicatorVars(c_train_label_s.reshape(-1, 1))
v_test_label = makeIndicatorVars(c_test_label.reshape(-1, 1))

In [32]:
v_train_label_resample_s = makeIndicatorVars(c_train_label_resample_s.reshape(-1, 1))

In [39]:
# write create simple linearLogistic Regression model
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(256, 82))
    tf_train_label = tf.placeholder(tf.float32, shape=(256,2))
    tf_validate_dataset = tf.constant(test_data_n.astype(np.float32)[:, :])
    tf_validate_label = tf.constant(v_test_label.astype(np.float32)[:])
    #tf_test_dataset1 = tf.constant(test_data_n.astype(np.float32)[:30000, :])
    #tf_test_label1 = tf.constant(test_label.astype(np.float32)[:30000])
    #tf_test_dataset2 = tf.constant(test_data_n.astype(np.float32)[30000:, :])
    #tf_test_label2 = tf.constant(test_label.astype(np.float32)[30000:])
    weights1 = tf.Variable(tf.truncated_normal([82, 10]))
    biases1 = tf.Variable(tf.zeros([10]))
    weights2 = tf.Variable(tf.truncated_normal([10, 2]))
    biases2 = tf.Variable(tf.zeros([2]))
    
    def acc(predict, label):
        #correct_prediction = tf.equal(predicted_label, tf_train_label)
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        predict_event = tf.reduce_sum(tf.argmax(predict, 1))
        label_event = tf.reduce_sum(tf.argmax(label, 1))
        true_positive = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 2), tf.int64))
        true_negative = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 0), tf.int64))
        false_positive = predict_event - true_positive 
        false_negative = label_event - true_positive
        return accuracy, false_positive, false_negative, true_positive, true_negative
    def ROC(FP, FN, TP, TN):
        TP_percent = TP / (TP + FN)
        FP_percent = FP / (FP + TN)
        return TP_percent, FP_percent
    
    def PRC(FP, FN, TP, TN):
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f_score = 2 * precision * recall / (precision + recall)
        return precision, recall, f_score
    
    def train_model(X):
        hidden = tf.nn.relu6(tf.matmul(X, weights1) + biases1)
        #hidden = tf.nn.dropout(hidden, 1.0)
        return tf.nn.softmax(tf.matmul(hidden, weights2) + biases2)
    
    def eval_model(X):
        hidden = tf.nn.relu6(tf.matmul(X, weights1) + biases1)
        return tf.nn.softmax(tf.matmul(hidden, weights2) + biases2)

    predicted_label = train_model(tf_train_dataset)
    #loss = tf.reduce_mean(tf.square(predicted_label - tf_train_label))
    loss = tf.reduce_mean(-tf.reduce_sum(tf_train_label * tf.log(predicted_label), reduction_indices=[1]))

    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 2500, 0.95, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    
    train_acc, _, _, _, _ = acc(predicted_label, tf_train_label)
    validate_acc, FP, FN, TP, TN = acc(eval_model(tf_validate_dataset), tf_validate_label)
    t_p, f_p = ROC(FP, FN, TP, TN)
    pre, rec, f_s = PRC(FP, FN, TP, TN)
    
    
    
    #train_loss = tf.reduce_mean(tf.abs(predicted_label - tf_train_label))
    #test_loss1 = tf.reduce_mean(tf.abs(model(tf_test_dataset1, weights, biases) - tf_test_label1))
    #test_loss2 = tf.reduce_mean(tf.abs(model(tf_test_dataset2, weights, biases) - tf_test_label2))
    #test_loss = (test_loss1 * 30000 + test_loss2 * (test_size - 30000)) / test_size
    #validate_loss = tf.reduce_mean(tf.abs(model(tf_validate_dataset, weights, biases) - tf_validate_label))


In [40]:
#train the model with stochastic gradient descent training
batch_size = 256
num_steps = 250000
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_n_resample_s[offset:(offset + batch_size), :]
        #print(batch_data.shape)
        batch_labels = v_train_label_resample_s[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_label : batch_labels}
        #session.run(predicted_label, feed_dict=feed_dict)
        _, l, _, r = session.run([predicted_label, loss, op, learning_rate], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print('Loss at step %d : learning_rate = %f, loss = %f' % (step, r, l))
            print(TP.eval(), FP.eval(), FN.eval(), TN.eval())
            print("precision = %f, recall = %f, f_score = %f" % (pre.eval(), rec.eval(), f_s.eval()))
            #print('Loss at step %d: LR %f MSE %f MAE %f VALIDATE MAE %f' % (step, r, l, train_loss.eval(feed_dict=feed_dict), validate_loss.eval()))
    #print(c_train_label_resample.reshape(-1, 1)[0:128, :])
    #print('test1, test2, test MAE: %.3f, %.3f, %.3f' % (test_loss1.eval(), test_loss2.eval(), test_loss.eval()))


Initialized
Loss at step 0 : learning_rate = 0.050000, loss = 1.581797
0 4047 421 24697
precision = 0.000000, recall = 0.000000, f_score = nan
Loss at step 1000 : learning_rate = 0.050000, loss = 0.059674
26 112 395 28632
precision = 0.188406, recall = 0.061758, f_score = 0.093023
Loss at step 2000 : learning_rate = 0.050000, loss = 0.035964
142 335 279 28409
precision = 0.297694, recall = 0.337292, f_score = 0.316258
Loss at step 3000 : learning_rate = 0.047500, loss = 0.060819
189 340 232 28404
precision = 0.357278, recall = 0.448931, f_score = 0.397895
Loss at step 4000 : learning_rate = 0.047500, loss = 0.063874
211 408 210 28336
precision = 0.340872, recall = 0.501188, f_score = 0.405769
Loss at step 5000 : learning_rate = 0.045125, loss = 0.065149
180 232 241 28512
precision = 0.436893, recall = 0.427553, f_score = 0.432173
Loss at step 6000 : learning_rate = 0.045125, loss = 0.053525
201 307 220 28437
precision = 0.395669, recall = 0.477435, f_score = 0.432723
Loss at step 7000 

In [40]:
c_train_label_resample[0:128]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

# Autoencoder

In [38]:
train_data_n.shape, train_label.shape

((116657, 82), (116657,))

In [70]:
# write create simple linear regression model
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(256, 82))
    tf_train_label = tf.placeholder(tf.float32, shape=(256, 82))
    tf_test_dataset = tf.constant(test_data_n.astype(np.float32))
   
   
    biases = tf.Variable(tf.zeros([1]))

    def model(X, w, b):
        hidden1 = tf.nn.relu6(tf.matmul(X, w['layer1']) + b['layer1'])
        hidden2 = tf.nn.tanh(tf.matmul(hidden1, w['layer2']) + b['layer2'])
        hidden3 = tf.nn.relu6(tf.matmul(hidden2, w['layer3']) + b['layer3'])
        hidden4 = tf.nn.tanh(tf.matmul(hidden3, w['layer4']) + b['layer4'])
        return hidden4, hidden2


    weights = {'layer1' : tf.Variable(tf.truncated_normal([82, 16])),
               'layer2' : tf.Variable(tf.truncated_normal([16, 2])),
               'layer3' : tf.Variable(tf.truncated_normal([2, 16])),
               'layer4' : tf.Variable(tf.truncated_normal([16, 82])),
               }
    biases = {'layer1' : tf.Variable(tf.zeros([16])),
              'layer2' : tf.Variable(tf.zeros([2])),
              'layer3' : tf.Variable(tf.zeros([16])),
              'layer4' : tf.Variable(tf.zeros([82])),
             }
    
    predicted_label, middle = model(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean((tf.square(predicted_label - tf_train_label)))
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.03
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 250, 0.95, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    #output the encoder from 
    _, hidden2_test = model(tf_test_dataset, weights, biases)

In [74]:
#train the model with stochastic gradient descent training
batch_size = 256
num_steps = 2500
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_n_s[offset:(offset + batch_size), :]
        #print(batch_data.shape)
        batch_labels = batch_data
        feed_dict = {tf_train_dataset : batch_data, tf_train_label : batch_labels}
        #session.run(predicted_label, feed_dict=feed_dict)
        _, l, _, r = session.run([predicted_label, loss, op, learning_rate], feed_dict=feed_dict)

        if (step % 100 == 0):
            print('Loss at step %d : learning_rate = %f, loss = %f' % (step, r, l))
            
    test_2d_point = hidden2_test.eval(feed_dict=feed_dict)
            

Initialized
Loss at step 0 : learning_rate = 0.030000, loss = 1.663830
Loss at step 100 : learning_rate = 0.030000, loss = 1.472725
Loss at step 200 : learning_rate = 0.030000, loss = 1.032295
Loss at step 300 : learning_rate = 0.028500, loss = 1.041697
Loss at step 400 : learning_rate = 0.028500, loss = 1.010043
Loss at step 500 : learning_rate = 0.027075, loss = 1.046748
Loss at step 600 : learning_rate = 0.027075, loss = 1.031405
Loss at step 700 : learning_rate = 0.027075, loss = 0.977066
Loss at step 800 : learning_rate = 0.025721, loss = 0.967030
Loss at step 900 : learning_rate = 0.025721, loss = 1.030428
Loss at step 1000 : learning_rate = 0.024435, loss = 0.988777
Loss at step 1100 : learning_rate = 0.024435, loss = 0.975351
Loss at step 1200 : learning_rate = 0.024435, loss = 0.977347
Loss at step 1300 : learning_rate = 0.023213, loss = 1.024334
Loss at step 1400 : learning_rate = 0.023213, loss = 0.956326
Loss at step 1500 : learning_rate = 0.022053, loss = 1.041771
Loss at 

In [75]:
test_2d_point.shape

(29165, 2)

In [86]:
a = test_2d_point[test_label <= 1.0]

In [87]:
a.shape

(421, 2)

In [90]:
a [:30]

array([[ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  0.99999988],
       [ 1.        ,  0.99980122],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  0.99999779],
       [ 1.        ,  0.99923235],
       [ 1.        ,  0.93946761],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ],
       [ 1.        ,